In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicting-citations-from-case-files1/citations_dataset.csv


In [11]:
!pip install pandas scikit-learn tqdm joblib
!pip install transformers torch accelerate
!pip install -q sentencepiece torch tqdm scikit-learn


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
import joblib
import ast
import torch
from tqdm.auto import tqdm



In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

Running on: cuda


In [15]:
df = pd.read_csv("/kaggle/input/predicting-citations-from-case-files1/citations_dataset.csv")
df.head()


,doc_body,citations_applied
0,Dharmendra Kumar vs The State Of Bihar Through...,"['Section 24', 'section 406', 'Section 138', '..."
1,Hira Lal Rattan Lal Etc. Etc vs State Of U.P. ...,"['1973 SCR (2) 502', 'Equivalent citations: 19..."
2,Naresh Singh Yadav vs The State Of Madhya Prad...,"['Section 482', 'Section 228', 'Section 397', ..."
3,Prof. Narendra Kumar Singh vs The State Of Bih...,['Section 15']
4,Raghvendra Dhari Singh vs The State Of Bihar T...,"['Section 24', 'section 406', 'Section 138', '..."


In [16]:
# Convert citation strings like "['A', 'B']" back to Python lists
df["citations_applied"] = df["citations_applied"].apply(lambda x: ast.literal_eval(x))

# Drop missing rows if any
df.dropna(subset=["doc_body", "citations_applied"], inplace=True)


In [17]:
# ---------------- Split into Train/Test ----------------
X_train, X_test, y_train, y_test = train_test_split(
    df["doc_body"], df["citations_applied"], test_size=0.2, random_state=42
)


In [18]:
# ---------------- Multi-label encoding ----------------
mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train)
y_test_bin = mlb.transform(y_test)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['(1964) 5 GUJLR 769', '(1964)10GLR769', '(1976)GLR497', '(1978) 114 ITR 523', '(1978) 91 MADLW 529', '(1978) CURTAXREP 311', '(1978) ILR 3 MAD 305', '(1978) KER LT 931', '(1982) 2 DMC 288', '(1982) 2 DMC 320', '(1982) 2 SCJ 238', '(1982) LS 60', '(1982) MAHLR 194', '(1982) MARRILJ 610', '(1982) MATLR 314', '(1983) 1 SCR 49 (SC)', '(1983) 1 SCWR 28', '(1983) 2 BOM CR 774', '(1983) 9 ALL LR 610', '(1983) ALL WC 798', '(1983) GUJ LH 1099', '(1984) JAB LJ 536', '(1984) MAH LJ 76', '(1984) MPLJ 1', '(1985)22TTJ(MUMBAI)494', '(1986) 1 APLJ 47', '(1986) 2 APLJ 37.2', '(1986) 2 COMLJ 249', '(1986) 2 CURLR 414', '(1986) 2 LAB LN 464', '(1986) 2 LAB LN 778', '(1986) 2 LABLJ 316', '(1986) 2 RENCJ 413', '(1986) 2 RENCR 283', '(1986) 2 SERVLR 481', '(1986) 2 SERVLR 743', '(1986) 2 SUPREME 358', '(1986) 3 SCJ 164', '(1986) 3 SCJ 63', '(1986) 3 SERVLJ 195', '(1986) 3 SUPREME 378', '(1986) 60 C

In [19]:
# ---------------- TF-IDF + RandomForest Model ----------------
from sklearn.pipeline import Pipeline

model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, stop_words='english')),
    ('clf', OneVsRestClassifier(RandomForestClassifier(
        n_estimators=150, random_state=42, n_jobs=-1
    )))
])

In [ ]:
# ---------------- Train Model ----------------
print("Training the model")
model.fit(X_train, y_train_bin)
print("Training complete!")


Training the model


In [ ]:
# ---------------- Evaluation ----------------
y_pred = model.predict(X_test_tfidf)
print("\n📊 Classification Report:")
print(classification_report(y_test_bin, y_pred, target_names=mlb.classes_))